In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [14]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_data = train_datagen.flow_from_directory('train',
                                              target_size=(128, 128),
                                              batch_size=32,
                                              class_mode='binary')

test_data = test_datagen.flow_from_directory('test',
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='binary')

Found 100 images belonging to 2 classes.


Found 200 images belonging to 2 classes.


In [15]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [16]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [19]:
model.fit(train_data,
          steps_per_epoch=train_data.samples // 32,
          epochs=100,
          validation_data=test_data,
          validation_steps=test_data.samples // 32)

Epoch 1/100
3/3 [==============================] - 2s 598ms/step - loss: 0.4641 - accuracy: 0.7941 - val_loss: 0.6916 - val_accuracy: 0.6250
Epoch 2/100
3/3 [==============================] - 2s 517ms/step - loss: 0.4968 - accuracy: 0.7647 - val_loss: 0.7162 - val_accuracy: 0.6146
Epoch 3/100
3/3 [==============================] - 1s 681ms/step - loss: 0.4065 - accuracy: 0.8382 - val_loss: 0.7199 - val_accuracy: 0.6250
Epoch 4/100
3/3 [==============================] - 1s 485ms/step - loss: 0.4345 - accuracy: 0.7647 - val_loss: 0.7434 - val_accuracy: 0.6198
Epoch 5/100
3/3 [==============================] - 1s 633ms/step - loss: 0.4732 - accuracy: 0.8088 - val_loss: 0.7644 - val_accuracy: 0.6094
Epoch 6/100
3/3 [==============================] - 1s 632ms/step - loss: 0.5021 - accuracy: 0.7206 - val_loss: 0.8015 - val_accuracy: 0.6042
Epoch 7/100
3/3 [==============================] - 1s 627ms/step - loss: 0.4912 - accuracy: 0.7500 - val_loss: 0.7491 - val_accuracy: 0.5833
Epoch 8/100
3

In [20]:
loss, accuracy = model.evaluate(test_data)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

7/7 [==============================] - 1s 87ms/step - loss: 1.0374 - accuracy: 0.6250
Test loss: 1.0374
Test accuracy: 0.6250


In [22]:
model.save('cat_dog_classifier.h5')

In [26]:
from tensorflow.keras.models import load_model
model = load_model('cat_dog_classifier.h5')
def preprocess_image(image_path):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(image_path, target_size=(128, 128))
    img = image.img_to_array(img)
    img = img / 255.0 
    img = np.expand_dims(img, axis=0) 
    return img
def predict_image(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)

    if prediction > 0.5:
        return 'Dog'
    else:
        return 'Cat'

image_path = 'test/dog/dog.4001.jpg'
result = predict_image(image_path)
print(f'The image is a {result}')
image_path


1/1 [==============================] - 0s 115ms/step
The image is a Dog


'test/dog/dog.4001.jpg'